# Обучение предиктивной модели для классификации типов искажений
Этот Notebook демонстрирует процесс обучения модели на базе ResNet50 для классификации изображений по типам (bad_print, brightness_contrast, clean, pixelation, not_document).

Мы используем предобученные веса ImageNet, аугментации для повышения robustness, mixed precision для экономии VRAM и мониторинг метрик (accuracy, loss).

Обучение включает early stopping, сохранение чекпоинтов и визуализацию результатов (графики, confusion matrix, ROC).

## Шаг 1: Импорт библиотек и настройка устройства
Импортируем необходимые библиотеки, определяем устройство (GPU/CPU).

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.amp import autocast, GradScaler
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, roc_curve, auc
import seaborn as sns
import os
from torch.nn.functional import softmax

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

## Шаг 2: Определение трансформаций данных
Настраиваем аугментации для train (флип, jitter, rotation) и базовые трансформации для val, с ресайзом на 256x362 и нормализацией ImageNet.

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((256, 362)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((256, 362)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## Шаг 3: Загрузка датасетов и лоадеров
Загружаем датасеты из папок train/val, создаём DataLoader с batch_size=64 для train и 32 для val.

In [ ]:
new_dataset_path = '/home/kudriavtcevroman-10/DocBank_Subset_Prediction_7000'
train_dataset = datasets.ImageFolder(root=os.path.join(new_dataset_path, 'train'), transform=train_transform)
val_dataset = datasets.ImageFolder(root=os.path.join(new_dataset_path, 'val'), transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

print(f"Train: {len(train_dataset)} изображений, Val: {len(val_dataset)} изображений")
print(f"Маппинг классов: {train_dataset.class_to_idx}")

## Шаг 4: Инициализация модели
Используем предобученную ResNet50, заменяем классификатор на 5-классовый, переносим на устройство.

In [ ]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_ftrs, 5)
)
model = model.to(device)
print("Модель инициализирована с предобученными весами ImageNet; fc адаптирован для 5 классов.")

## Шаг 5: Настройка оптимизатора, scheduler и early stopping
Оптимизатор AdamW, loss CrossEntropy, scheduler CosineAnnealingLR, mixed precision с GradScaler, early stopping по val accuracy.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)
scaler = GradScaler()
patience = 5
early_stop_counter = 0
best_val_acc = 0.0

train_losses, val_losses = [], []
train_accs, val_accs = [], []

num_epochs = 100

## Шаг 6: Цикл обучения
Обучаем модель с mixed precision, мониторим loss и accuracy, применяем scheduler и early stopping по val accuracy.

In [ ]:
for epoch in range(num_epochs):
    model.train()
    train_loss, train_correct = 0.0, 0
    for inputs, labels in tqdm(train_loader, desc=f"Train Epoch {epoch+1}"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        with autocast(device_type='cuda'):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item() * inputs.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()

    avg_train_loss = train_loss / len(train_dataset)
    avg_train_acc = train_correct / len(train_dataset)
    train_losses.append(avg_train_loss)
    train_accs.append(avg_train_acc)

    model.eval()
    val_loss, val_correct = 0.0, 0
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc=f"Val Epoch {epoch+1}"):
            inputs, labels = inputs.to(device), labels.to(device)
            with autocast(device_type='cuda'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            val_correct += (outputs.argmax(1) == labels).sum().item()

    avg_val_loss = val_loss / len(val_dataset)
    avg_val_acc = val_correct / len(val_dataset)
    val_losses.append(avg_val_loss)
    val_accs.append(avg_val_acc)

    scheduler.step()
    print(f"Эпоха {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Train Acc: {avg_train_acc:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {avg_val_acc:.4f}")

    if avg_val_acc > best_val_acc:
        best_val_acc = avg_val_acc
        torch.save(model.state_dict(), "prediction_model.pth")
        print("→ Новая лучшая модель сохранена!")
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping: нет улучшения.")
            break

## Шаг 7: Сохранение модели
Сохраняем обученные веса для дальнейшего использования в инференсе.

In [ ]:
torch.save(model.state_dict(), '/home/kudriavtcevroman-10/prediction_model.pth')
print("Финальная модель сохранена как prediction_model.pth")

## Шаг 8: Визуализация метрик обучения
Строим графики loss и accuracy для анализа сходимости и проверки на переобучение.

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 10))
axs[0].plot(range(1, len(train_losses)+1), train_losses, label='Train Loss', marker='o')
axs[0].plot(range(1, len(val_losses)+1), val_losses, label='Val Loss', marker='o')
axs[0].set_title('Loss over Epochs')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[0].legend()
axs[0].grid(True)

axs[1].plot(range(1, len(train_accs)+1), train_accs, label='Train Acc', marker='o')
axs[1].plot(range(1, len(val_accs)+1), val_accs, label='Val Acc', marker='o')
axs[1].set_title('Accuracy over Epochs')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Accuracy')
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.savefig('prediction_model_metrics.png')
plt.show()
print("Графики метрик сохранены как prediction_model_metrics.png")

## Шаг 9: Confusion matrix и ROC-кривые
Вычисляем confusion matrix и ROC для оценки качества классификации на val.

In [ ]:
all_preds, all_labels, all_probs = [], [], []
model.eval()
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        probs = softmax(outputs, dim=1)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

cm = confusion_matrix(all_labels, all_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.savefig('prediction_model_confusion_matrix.png')
plt.show()

In [ ]:
all_probs = np.array(all_probs)
plt.figure()
for i in range(5):
    fpr, tpr, _ = roc_curve(np.array(all_labels) == i, all_probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Класс {i} (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривые')
plt.legend(loc="lower right")
plt.savefig('prediction_model_roc_curves.png')
plt.show()